In [ ]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [ ]:
print('EXS3.DE','EXS1.DE','EXS2.DE','EXV1.DE','EXH2.DE','EXH3.DE','EXH5.DE','EXH7.DE','EXV3.DE',
               'EXV9.DE','EXV4.DE','IQQC.DE','IQQP.DE','IQQQ.DE','INRG.MI','L0CK.DE','XAIX.DE','2B70.DE',
               'QDVX.DE','QDVW.DE','C005.DE')

In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.linear_model import LinearRegression
from scipy import stats
from scipy.stats import shapiro
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from datetime import datetime, timedelta
today_val = datetime.now()
today_val.strftime("%Y-%m-%d")
yesterday_val = (today_val-timedelta(days=1)).strftime("%Y-%m-%d")

import datetime
import time

In [ ]:
def ticker_var(start):
    tickerData = yf.Ticker(tickerSymbol)
    tickerDf = tickerData.history(start=start, end=yesterday_val, interval="1wk", auto_adjust = False)
    tickerDf = tickerDf.reset_index()
    tickerDf = tickerDf.dropna(subset=['Close'])
    tickerDf = tickerDf.reset_index()
    del tickerDf['index']
    return tickerDf   

In [ ]:
def figures():
    for start in set_list:
        tickerDf = ticker_var(start)
        print("Set Date: ", start,' | Length: ',len(tickerDf))

        if int(tickerDf.loc[0,'Date'].strftime("%Y-%m-%d %H:%M:%S")[0:4]) > int(start[0:4]):
            print('Not enough data! [1]')
            continue

        df = [tickerDf["Date"], tickerDf["Close"]]
        headers = ["Date", "Close"]
        df = pd.concat(df, axis=1, keys=headers)
        df.set_index('Date', inplace=True)

        df_log = np.log(df)

        week_diff = tickerDf['Date'].diff()
        week_diff = week_diff[1:len(week_diff)-1]
        week_diff = week_diff.drop_duplicates()

        if len(week_diff) > 1:
            print("Data incomplete!")
            prin

        set = len(tickerDf[tickerDf['Date'] < '2022-01-01 00:00:00'])-1

        first_date = tickerDf.loc[0,'Date']
        first_unix_str= first_date.strftime("%Y-%m-%d %H:%M:%S")
        first_unix = datetime.datetime.timestamp(datetime.datetime.strptime(first_unix_str, "%Y-%m-%d %H:%M:%S"))
        last_date = tickerDf.loc[len(tickerDf)-1,'Date']
        last_unix_str= last_date.strftime("%Y-%m-%d %H:%M:%S")
        last_unix = datetime.datetime.timestamp(datetime.datetime.strptime(last_unix_str, "%Y-%m-%d %H:%M:%S"))

        if first_unix > 1588347863:
            print("Not enough data! [2]")
            prin

        if last_unix < 1588347863:
            print("No more date!")
            prin

        if len(tickerDf[0:set]) < 50:
            print("Not enough data! {3}")
            prin

        x = np.log(df.values)

        train = x[0:set]
        test = x[set:len(tickerDf)]
        predictions = []

        import itertools
        import warnings
        warnings.filterwarnings('ignore')

        combinations = pd.DataFrame([])

        p=q=range(0,3)
        d = range(1,3)
        pdq = list(itertools.product(p,d,q))
        i = 0
        for param in pdq:
            try:
                model_arima = ARIMA(train,order=param)
                model_arima_fit = model_arima.fit()
                combinations.loc[i,'p,d,q'] = str(param)
                combinations.loc[i,'AIC'] = model_arima_fit.aic

            except:
                continue

            i = i + 1

        combinations = combinations.sort_values(by='AIC')
        s = combinations.iloc[0,0]
        s = s.replace('(', '')
        s = s.replace(')', '')
        param_opt = tuple(map(int, s.split(', ')))

        model_arima = ARIMA(train,order=param_opt)
        model_arima_fit = model_arima.fit()
        predictions = model_arima_fit.forecast(steps=len(tickerDf)-set)[0]

        test_x = list(range(set, set+len(test)))
        X = np.array(test_x).reshape((-1, 1))
        Y = predictions.reshape((-1, 1))
        if len(X) == len(Y):
            reg = LinearRegression().fit(X, Y)
            score = reg.score(X, Y)
            coef = reg.coef_[0][0]
            intercept = reg.intercept_[0]
            Y_pred = reg.predict(X)
            std_2 = np.std(Y_pred)
        else:
            print("Fehler")

        i = 0
        tickerDf['SMA40W'] = tickerDf['Close'].rolling(window = 40).mean()
        tickerDf['SMA18W'] = tickerDf['Close'].rolling(window = 18).mean()
        std = tickerDf.loc[0:set,'Close'].std()

        while i < len(tickerDf):

            tickerDf.loc[i,'Est_Value_Normal'] = (coef * (i+1)) + intercept

            i = i + 1

        tickerDf['Est_Value_Normal'] = np.exp(tickerDf['Est_Value_Normal'])

        Residual_Sum = ((tickerDf['Close']-tickerDf['Est_Value_Normal'])).sum()
        Sum_Rows = len(tickerDf['Close'])
        Adaption = Residual_Sum/Sum_Rows 
        tickerDf['Est_Value_Normal'] = tickerDf['Est_Value_Normal'] + Adaption

        tickerDf['Est_Value_Max_1'] = tickerDf['Est_Value_Normal'] + std
        tickerDf['Est_Value_Min_1'] = tickerDf['Est_Value_Normal']-+ std
        tickerDf['Est_Value_Max_2'] = tickerDf['Est_Value_Normal'] + 2*std
        tickerDf['Est_Value_Min_2'] = tickerDf['Est_Value_Normal'] - 2*std

        KPI = round((tickerDf.loc[len(tickerDf)-1,'Close'] - tickerDf.loc[len(tickerDf)-1,'Est_Value_Normal']) / std ,2)
        Trend_max_1 = str(round(tickerDf.loc[len(tickerDf)-1,'Est_Value_Max_1'],2))
        Trend_min_1 = str(round(tickerDf.loc[len(tickerDf)-1,'Est_Value_Min_1'],2))

        fig = plt.figure(figsize = (18,12))
        ax = fig.add_subplot()
        try:
            plt.title(stock.info['longName']+"\n["+str(start[0:4])+"]", fontsize = 20)
        except Exception:
                pass

        plt.xlabel('Date', fontsize = 20)
        plt.ylabel('Price', fontsize = 20)

        ax.text(0.05, 0.75, 'KPI: ' + str(KPI)
                + '\n\nTrend max 1: ' + Trend_max_1+ '\n\nTrend min 1: ' + Trend_min_1,
            verticalalignment='bottom', horizontalalignment='left', transform=ax.transAxes,  color='black', fontsize=20)

        ax.text(0.75, 0.05, 'Price: ' + str(round(tickerDf.loc[len(tickerDf)-1,'Close'],2)),
            verticalalignment='bottom', horizontalalignment='left', transform=ax.transAxes,  color='black', fontsize=20)

        ax.yaxis.set_label_position("right")
        ax.yaxis.tick_right()
        mark = tickerDf['Date'][set]
        plt.axvline(x = mark, color='red', ls='--', lw=1)
        mark2 = tickerDf['Close'][len(tickerDf)-1]
        ax.axhline(mark2, color='red', ls='--', lw=1)

        plt.plot(tickerDf['Date'],tickerDf[['Close']], color = "Blue")
        plt.plot(tickerDf['Date'],tickerDf[['Est_Value_Normal']], color = "Black")                                   
        plt.plot(tickerDf['Date'],tickerDf[['Est_Value_Max_1']], color = "Black",ls='--')  
        plt.plot(tickerDf['Date'],tickerDf[['Est_Value_Min_1']], color = "Black",ls='--')  
        plt.plot(tickerDf['Date'],tickerDf[['Est_Value_Max_2']], color = "grey",ls='--')  
        plt.plot(tickerDf['Date'],tickerDf[['Est_Value_Min_2']], color = "grey",ls='--')

        plt.show(fig)

In [ ]:
def stock_info():
    information_df = pd.DataFrame([])
    try:
        information_df.loc[0,'Name'] = stock.info['longName']
    except Exception:
        pass
    try:
        information_df.loc[0,'Sector'] = stock.info['sector']
    except Exception:
        pass
    try:
        information_df.loc[0,'200D Moving Average'] = stock.info['twoHundredDayAverage']
    except Exception:
        pass
    try:
        information_df.loc[0,'Annual Dividend Yield %'] = round(stock.info['trailingAnnualDividendYield']*100,2)
    except Exception:
        pass
    try:
        information_df.loc[0,'Dividend'] = stock.info['dividendRate']
    except Exception:
        pass
    try:
        information_df.loc[0,'Free Float [%]'] = round(stock.info['shortPercentOfFloat']*100,2)
    except Exception:
        pass
    try:
        information_df.loc[0,'Current PE'] = stock.info['trailingPE']
    except Exception:
        pass
    try:
        information_df.loc[0,'Forward PE'] = stock.info['forwardPE']
    except Exception:
        pass
    try:
        information_df.loc[0,'Margin [%]'] = round(stock.info['profitMargins']*100,2)
    except Exception:
        pass
    try:
        information_df.loc[0,'Shortage-Ratio [%]'] = round(stock.info['sharesPercentSharesOut']*100,2)
    except Exception:
        pass
    try:
        information_df.loc[0,'Percentage Held Institutions [%]'] = round(stock.info['heldPercentInstitutions']*100,2)
    except Exception:
        pass
    return information_df  

In [ ]:
ticker_text = widgets.Text(placeholder='Tickername')
widgets.VBox([ticker_text])

In [ ]:
button = widgets.Button(description="Click To Start!")
output = widgets.Output()
display(button, output)

In [ ]:
def on_button_clicked(b):
       
    with output:
        global start,tickerSymbol,set_list
        tickerSymbol = ticker_text.value
        print("\nAnalysis started for", tickerSymbol,"\n")   
        stock = yf.Ticker(tickerSymbol)
        print(stock.info['longName'])
        set_list = ['2010-1-1','2013-1-1','2016-1-1','2018-1-1','2020-4-1']

        figures()
        information_df = stock_info()
        information_df       
    
button.on_click(on_button_clicked)